<a href="https://colab.research.google.com/github/camilagoncalves1/generating_routes/blob/main/generating_routes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importação de Bibliotecas

In [ ]:
!pip install googlemaps folium polyline geopy requests matplotlib

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=a80f10b93493b77745e73be15b6077501c8e960d221ebe50022f026176dc7fa1
  Stored in directory: /root/.cache/pip/wheels/4c/6a/a7/bbc6f5c200032025ee655deb5e163ce8594fa05e67d973aad6
Successfully built googlemaps


## Configuração e Mapeamento de Ruas

In [ ]:
import folium
from geopy.geocoders import Nominatim
import time
import polyline as polyline_module
import requests

def processar_itinerario_longo(itinerario_texto):
    """
    Processa itinerários longos removendo duplicatas e similares
    """
    linhas = [linha.strip() for linha in itinerario_texto.split('\n') if linha.strip()]

    # Filtrar e limpar linhas que parecem endereços
    enderecos = []
    for linha in linhas:
        # Critérios mais flexíveis para identificar endereços
        # Inclui nomes de ruas, avenidas, rodovias, etc. e também pontos de referência com números
        if any(keyword in linha.upper() for keyword in ['AVENIDA', 'RUA', 'ESTRADA', 'RODOVIA', 'BR-', 'RJ-', 'TERMINAL', 'TREVO', 'PLATAFORMA']) or any(char.isdigit() for char in linha):
            # Limpar o endereço
            endereco_limpo = limpar_endereco(linha)
            if endereco_limpo and endereco_limpo not in enderecos:
                enderecos.append(endereco_limpo)

    print(f"Endereços únicos encontrados: {len(enderecos)}")
    return enderecos

def limpar_endereco(endereco):

    # Remover informações desnecessárias como " - Rio De Janeiro Brasil", CEPs no final
    partes = endereco.split(' - ')
    endereco_principal = partes[0].strip()

    # Remover números de 5 dígitos (provavelmente CEP) no final do endereço principal
    palavras = endereco_principal.split()
    if palavras and palavras[-1].isdigit() and len(palavras[-1]) == 5:
        endereco_principal = ' '.join(palavras[:-1])

    # Padronizar "Rio De Janeiro" para "Rio de Janeiro"
    endereco_principal = endereco_principal.replace('Rio De Janeiro', 'Rio de Janeiro')

    # Adicionar cidade e estado para ajudar na geocodificação, se não estiver presente
    if 'Rio de Janeiro' not in endereco_principal and 'RJ' not in endereco_principal:
        endereco_limpo = f"{endereco_principal}, Rio de Janeiro, RJ"
    else:
        endereco_limpo = endereco_principal

    # Remover espaços extras
    endereco_limpo = ' '.join(endereco_limpo.split())

    return endereco_limpo

def agrupar_enderecos_similares(enderecos):
    """
    Agrupa endereços muito similares para reduzir pontos
    """
    grupos = []
    enderecos_usados = set()

    for endereco in enderecos:
        if endereco in enderecos_usados:
            continue

        # Encontrar endereços similares
        similares = [endereco]
        endereco_base = extrair_endereco_base(endereco)

        for outro in enderecos:
            if outro != endereco and outro not in enderecos_usados:
                outro_base = extrair_endereco_base(outro)
                # Comparar bases de endereço para agrupar
                if endereco_base == outro_base:
                    similares.append(outro)
                    enderecos_usados.add(outro)

        # Escolher o endereço mais representativo do grupo (geralmente o mais simples/curto)
        if len(similares) == 1:
            grupos.append(similares[0])
        else:
            # Priorizar endereços mais curtos
            similares_ordenados = sorted(similares, key=lambda x: len(x))
            grupos.append(similares_ordenados[0]) # Pega o mais curto do grupo

        enderecos_usados.add(endereco)

    print(f"Endereços após agrupamento: {len(grupos)}")
    return grupos


def extrair_endereco_base(endereco):
    """
    Extrai a parte base do endereço (removendo números que parecem CEPs ou números de prédio no final)
    e informações de cidade/estado no final.
    """
    partes = endereco.split(',')
    endereco_principal = partes[0].strip()

    # Remover números que parecem CEPs (5 dígitos) ou números de prédio no final
    palavras = endereco_principal.split()
    palavras_sem_numeros = []
    for p in palavras:
        # Mantém se não for um número de 5 dígitos (CEP) ou um número sozinho no final da string original
        if not (p.isdigit() and (len(p) == 5 or (palavras.index(p) == len(palavras) - 1 and len(palavras) > 1 and p.isdigit()))):
             palavras_sem_numeros.append(p)

    return ' '.join(palavras_sem_numeros).strip()


def selecionar_pontos_estrategicos(enderecos, max_pontos=20): # Aumentando um pouco o limite para 20
    """
    Seleciona pontos estratégicos ao longo do itinerário
    """
    if len(enderecos) <= max_pontos:
        return enderecos

    # Selecionar pontos distribuídos uniformemente, garantindo o primeiro e o último
    indices = set()
    indices.add(0) # Primeiro ponto
    indices.add(len(enderecos) - 1) # Último ponto

    # Distribuir pontos intermediários
    passo = (len(enderecos) - 1) / (max_pontos - 1)
    for i in range(1, max_pontos - 1):
        indices.add(int(i * passo))

    indices_ordenados = sorted(list(indices))
    pontos_selecionados = [enderecos[i] for i in indices_ordenados]


    print(f"Pontos estratégicos selecionados: {len(pontos_selecionados)}")
    return pontos_selecionados


def tracar_rota_longa(itinerario_texto):
    """
    Traça rotas para itinerários longos como o do Moovit
    """
    print("=== PROCESSANDO ITINERÁRIO LONGO ===")

    # Processar endereços
    enderecos = processar_itinerario_longo(itinerario_texto)

    if not enderecos:
        print("❌ Nenhum endereço válido encontrado")
        return None

    # Agrupar endereços similares
    enderecos_agrupados = agrupar_enderecos_similares(enderecos)

    # Selecionar pontos estratégicos
    waypoints = selecionar_pontos_estrategicos(enderecos_agrupados, max_pontos=20) # Usando 20 pontos

    print("\nWaypoints selecionados para geocodificação:")
    for i, wp in enumerate(waypoints):
        print(f"  {i+1}: {wp}")

    # Geocodificar
    geolocator = Nominatim(user_agent="rota_longa_moovit_v3") # Mudando user_agent novamente
    coordenadas = []
    waypoints_validos = []

    print(f"\nGeocodificando {len(waypoints)} waypoints...")

    for i, wp in enumerate(waypoints):
        print(f"  {i+1}/{len(waypoints)}: {wp[:50]}...")
        try:
            # Tentativas de geocodificação
            location = geolocator.geocode(wp, timeout=20)
            if not location:
                 # Tentar sem a cidade/estado se a primeira tentativa falhar
                 endereco_sem_cidade = wp.split(',')[0]
                 location = geolocator.geocode(endereco_sem_cidade + ", Rio de Janeiro, RJ", timeout=20)

            if location:
                coordenadas.append((location.latitude, location.longitude))
                waypoints_validos.append(wp)
                print(f"    ✅ {location.latitude:.4f}, {location.longitude:.4f}")
            else:
                print(f"    ❌ Não encontrado")
        except Exception as e:
            print(f"    ⚠ Erro: {e}")

        time.sleep(1.5)  # Respeitar rate limit

    if len(coordenadas) < 2:
        print(f"\n❌ Apenas {len(coordenadas)} waypoint(s) encontrado(s). Não é possível traçar a rota.")
        return None

    # Criar mapa
    mapa2 = folium.Map(
        location=coordenadas[0],
        zoom_start=11,
        tiles='OpenStreetMap'
    )

    # Adicionar marcadores
    for i, (coord, wp) in enumerate(zip(coordenadas, waypoints_validos)):
        nome_curto = wp.split(',')[0][:30] + "..." if len(wp.split(',')[0]) > 30 else wp.split(',')[0]

        if i == 0:
            icon_color, icon_name = 'green', 'play'
            popup_text = f'<b>🚌 INÍCIO</b><br>{nome_curto}'
        elif i == len(coordenadas) - 1:
            icon_color, icon_name = 'red', 'stop'
            popup_text = f'<b>🏁 FIM</b><br>{nome_curto}'
        else:
            icon_color, icon_name = 'blue', 'flag'
            popup_text = f'<b>📍 Ponto {i+1}</b><br>{nome_curto}'

        folium.Marker(
            coord,
            popup=popup_text,
            tooltip=f'Ponto {i+1}: {nome_curto}',
            icon=folium.Icon(color=icon_color, icon=icon_name, prefix='fa')
        ).add_to(mapa2)

    # Traçar rotas em segmentos
    print(f"\nTraçando rotas entre {len(coordenadas)} pontos...")
    for i in range(len(coordenadas) - 1):
        try:
            url = f"http://router.project-osrm.org/route/v1/driving/{coordenadas[i][1]},{coordenadas[i][0]};{coordenadas[i+1][1]},{coordenadas[i+1][0]}?overview=full"
            response = requests.get(url, timeout=15)

            if response.status_code == 200:
                data = response.json()
                if data['code'] == 'Ok':
                    polyline_str = data['routes'][0]['geometry']
                    pontos = polyline_module.decode(polyline_str)
                    folium.PolyLine(
                        [(lat, lng) for lat, lng in pontos],
                        color='blue',
                        weight=5,
                        opacity=0.7,
                        popup=f'Trecho {i+1}'
                    ).add_to(mapa2)
                    print(f"  ✅ Trecho {i+1}: Rota calculada")
                    continue
        except Exception as e:
            print(f"  ⚠ Trecho {i+1}: Erro na API - {e}")

        # Fallback: linha reta
        folium.PolyLine(
            [coordenadas[i], coordenadas[i+1]],
            color='red',
            weight=2,
            opacity=0.5,
            dash_array='5, 5',
            popup=f'Trecho {i+1} (aproximado)'
        ).add_to(mapa2)
        print(f"  ⚠ Trecho {i+1}: Rota aproximada")

    # Adicionar informações do itinerário
    info_html = f"""
    <div style="
        position: fixed;
        top: 10px;
        left: 50px;
        width: 300px;
        background-color: white;
        border: 2px solid grey;
        z-index: 9999;
        font-size: 12px;
        padding: 10px;
        border-radius: 5px;
    ">
    <b>LINHA 544P - ITINERÁRIO RESUMIDO</b><br>
    Pontos: {len(coordenadas)} de {len(enderecos)} originais<br>
    <span style="color: green">●</span> Início |
    <span style="color: red">●</span> Fim |
    <span style="color: blue">●</span> Intermediários
    </div>
    """
    mapa2.get_root().html.add_child(folium.Element(info_html))

    mapa2.save('rota_linha_544P.html')

    print(f"\n✅ Mapa salvo como 'rota_linha_544P.html'")
    print(f"📊 Estatísticas:")
    print(f"   - Endereços originais: {len([l for l in itinerario_texto.split('\n') if l.strip()])}")
    print(f"   - Endereços únicos: {len(enderecos)}")
    print(f"   - Pontos no mapa: {len(coordenadas)}")

    return mapa2

# EXECUÇÃO PRINCIPAL
if __name__ == "__main__":
    # Cole todo o itinerário longo aqui
    itinerario_longo = """ Av. Júlio Maria,
    Av. Dr. Coutinho,
    Av. Ayrton Senna,
    Av. Caravelas,
    Av. José Elias Rabha,
    Trevo da Sapinhatuba II,
    Rodovia Procurador Haroldo Fernandes Duarte (BR-101),
    Terminal Rodoviário de Praia Brava,
    Rodovia Procurador Haroldo Fernandes Duarte (BR-101),
    Rua do Areal,
    Av. Francisco Magalhães de Castro,
    Rua Sete de Abril, Rua Bandeirantes,
    Rua Getúlio Vargas,
    Rua Júlio Maria,
    Rua Sete de Abril,
    Rua da Limeira,
    Rua dos Bandeirantes (Campo da Gringa) """

    print("Processando itinerário...")
    mapa2 = tracar_rota_longa(itinerario_longo)

    if mapa2:
        print("\n🎉 Mapa da Linha 544P gerado com sucesso!")
        print("📁 Abra o arquivo 'rota_linha_544P.html' no seu navegador")
    else:
        print("\n❌ Não foi possível gerar o mapa")

Processando itinerário...
=== PROCESSANDO ITINERÁRIO LONGO ===
Endereços únicos encontrados: 10
Endereços após agrupamento: 9

Waypoints selecionados para geocodificação:
  1: Trevo da Sapinhatuba II,, Rio de Janeiro, RJ
  2: Rodovia Procurador Haroldo Fernandes Duarte (BR-101),, Rio de Janeiro, RJ
  3: Terminal Rodoviário de Praia Brava,, Rio de Janeiro, RJ
  4: Rua do Areal,, Rio de Janeiro, RJ
  5: Rua Sete de Abril,, Rio de Janeiro, RJ
  6: Rua Getúlio Vargas,, Rio de Janeiro, RJ
  7: Rua Júlio Maria,, Rio de Janeiro, RJ
  8: Rua da Limeira,, Rio de Janeiro, RJ
  9: Rua dos Bandeirantes (Campo da Gringa), Rio de Janeiro, RJ

Geocodificando 9 waypoints...
  1/9: Trevo da Sapinhatuba II,, Rio de Janeiro, RJ...
    ❌ Não encontrado
  2/9: Rodovia Procurador Haroldo Fernandes Duarte (BR-10...
    ❌ Não encontrado
  3/9: Terminal Rodoviário de Praia Brava,, Rio de Janeir...
    ❌ Não encontrado
  4/9: Rua do Areal,, Rio de Janeiro, RJ...
    ✅ -23.0196, -44.5333
  5/9: Rua Sete de Abril

In [ ]:
mapa2